In [ ]:
# Important! This way you can navigate through $home an get the path to the NAS data
from pathlib import Path

import pandas as pd
from tqdm import tqdm

home = str(Path.home())
home

In [2]:
import pandas as pd
from copy import deepcopy, copy
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
tqdm.pandas()
from sklearn.model_selection import train_test_split
import numpy as np
from codetiming import Timer
import pyleetspeak
# print(f"Pyleetspeak version: {pyleetspeak.__version__}")
print(f"Spacy version: {spacy.__version__}")


from pyleetspeak.pyleetspeak import WordCamouflage_Augmenter

import warnings
warnings.filterwarnings("ignore")

2023-06-04 08:11:00.368829: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-04 08:11:00.654352: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-04 08:11:00.710962: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-04 08:11:01.753536: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

Spacy version: 3.4.3


[nltk_data] Downloading package stopwords to /home/alvaro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/alvaro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Functions

In [3]:
def make_docs(data_tuples, nlp, labels):
    """_summary_
    """
    docs = []
    for text, label in tqdm(nlp.pipe(data_tuples, as_tuples=True), total = len(data_tuples), desc = "Making docs"):
        doc = nlp(text)

        for l in labels:
            # Hay que hacer todos los labels
            doc.cats[l] = label == l   

        # put them into a nice list
        docs.append(doc)
    
    return docs

def leet_data(text, generator):
    """_summary_

    Args:
        text (_type_): _description_
        generator (_type_): _description_

    Returns:
        _type_: _description_
    """
    NER_data, ori_data = generator.generate_data(
            sentence=text
            # important_kws = [r"\bpfizer\b", r"control\b", r"vacuna\b", r"vaccines\b"],
        )

    leet_text, _ =  NER_data[0]
    return leet_text


# function to create dataframes witrh a percentage leet tweets
def create_leet_df(df_ori, frac, generator, column_to_leet):
    """_summary_

    Args:
        df (_type_): _description_
        frac (_type_): _description_
        generator (_type_): _description_
        column_to_leet (_type_): _description_

    Returns:
        _type_: _description_
    """
    # Create a copy of the original dataframe for saving the leeted version
    df_leeted = copy(df_ori)
    df_leeted["Camouflaged"] = False
    
    # Extract fraction to leet
    df_to_leet = df_leeted.sample(frac=frac, random_state=42)

    # Leet the tweets
    df_to_leet[column_to_leet] = df_to_leet[column_to_leet].progress_apply(leet_data,  generator=generator_EN)
    df_to_leet["Camouflaged"] = True

    # count nan values in df_train_offen_to_leet["tweet"] column
    print(f"Nan values in '{column_to_leet}' column: ", df_to_leet[column_to_leet].isna().sum())

    # Substitute the original rows by the leeted version
    cols = list(df_leeted.columns) 
    df_leeted.loc[df_leeted.index.isin(df_to_leet.index), cols] = df_to_leet[cols]    

    display( df_leeted.groupby("Camouflaged").count() ) 
    
    return df_leeted


def create_augmented_df(df_ori, frac, generator):
    """_summary_

    Args:
        df_ori (_type_): _description_
        frac (_type_): _description_
        generator (_type_): _description_

    Returns:
        _type_: _description_
    """
    
    # Create a copy of the original dataframe for saving the leeted version
    df_augmented = copy(df_ori)
    df_augmented["Camouflaged"] = False
    
    # Extract different fraction to leet 
    df_to_augment = df_augmented.sample(frac=frac, random_state=42)
    df_to_augment_2 = df_augmented.sample(frac=frac, random_state=4)
    df_to_augment_3 = df_augmented.sample(frac=frac, random_state=12)


    # Leet the tweets
    df_to_augment["tweet"] = df_to_augment["tweet"].progress_apply(leet_data,  generator=generator)
    df_to_augment["Camouflaged"] = True

    df_to_augment_2["tweet"] = df_to_augment_2["tweet"].progress_apply(leet_data,  generator=generator)
    df_to_augment_2["Camouflaged"] = True

    df_to_augment_3["tweet"] = df_to_augment_3["tweet"].progress_apply(leet_data,  generator=generator)
    df_to_augment_3["Camouflaged"] = True



    # count nan values in df_train_offen_to_leet["tweet"] column
    print("Nan values in df_train_offen_to_leet['tweet'] column: ", df_to_augment["tweet"].isna().sum())
    print("Nan values in df_train_offen_to_leet['tweet'] column: ", df_to_augment_2["tweet"].isna().sum())
    print("Nan values in df_train_offen_to_leet['tweet'] column: ", df_to_augment_3["tweet"].isna().sum())

    # Add the augmented tweets to the original dataframe
    df_augmented = pd.concat([df_augmented, df_to_augment, df_to_augment_2, df_to_augment_3], ignore_index=True)
    # suffle the dataframe
    df_augmented = df_augmented.sample(frac=1, random_state=42)


    display( df_augmented.groupby("Camouflaged").count() ) 
    
    return df_augmented



# Create a function to save pandas dataframe to spacy binary file
def pd_2_spacy(df_train, df_dev, df_test, train_output_path, dev_output_path, test_output_path, labels, lang="en"):
    """_summary_

    Args:
        df_train (_type_): _description_
        df_dev (_type_): _description_
        df_test (_type_): _description_
        train_output_path (_type_): _description_
        dev_output_path (_type_): _description_
        test_output_path (_type_): _description_
        lang (str, optional): _description_. Defaults to "en".

    Returns:
        _type_: _description_
    """
    # Spact empty model
    nlp = spacy.blank("en")

    if df_train is not None:
        # tuple of tuples. Each nested tuple is (Tweet, Label)
        train_data_tuples = tuple(df_train.iloc[:, [0,1]].itertuples(index=False, name=None))
                
        # Make spacy DocBin
        train_docs = make_docs(train_data_tuples, nlp, labels)

        # Make outpath directory with Pathlib
        Path(train_output_path).parent.mkdir(parents=True, exist_ok=True)

        # save to binary file 
        train_doc_bin = DocBin(docs=train_docs)
        train_doc_bin.to_disk(train_output_path)
        print(f"Processed Train {len(train_data_tuples)} documents: {train_output_path}")        

    if df_dev is not None:
        dev_data_tuples = tuple(df_dev.iloc[:, [0,1]].itertuples(index=False, name=None))
        dev_docs = make_docs(dev_data_tuples, nlp, labels)
        Path(dev_output_path).parent.mkdir(parents=True, exist_ok=True)
        dev_doc_bin = DocBin(docs=dev_docs)
        dev_doc_bin.to_disk(dev_output_path)
        print(f"Processed Dev {len(dev_data_tuples)} documents: {dev_output_path}")

    if df_test is not None:
        test_data_tuples = tuple(df_test.iloc[:, [0,1]].itertuples(index=False, name=None))
        test_docs = make_docs(test_data_tuples, nlp, labels)
        Path(test_output_path).parent.mkdir(parents=True, exist_ok=True)   

        test_doc_bin = DocBin(docs=test_docs)
        test_doc_bin.to_disk(test_output_path)
        print(f"Processed Test {len(test_data_tuples)} documents: {test_output_path}")    


def leet_data_augmenter(text, augmenter):
    """_summary_

    Args:
        text (_type_): _description_
        generator (_type_): _description_

    Returns:
        _type_: _description_
    """
    leet_text = augmenter.transform(
            text
        )
    return leet_text


# function to create dataframes witrh a percentage leet tweets
def create_leet_augmenter_df(df_ori, frac, augmenter, column_to_leet):
    """_summary_

    Args:
        df (_type_): _description_
        frac (_type_): _description_
        generator (_type_): _description_
        column_to_leet (_type_): _description_

    Returns:
        _type_: _description_
    """
    # Create a copy of the original dataframe for saving the leeted version
    df_leeted = copy(df_ori)
    df_leeted["Camouflaged"] = False
    
    # Extract fraction to leet
    df_to_leet = df_leeted.sample(frac=frac, random_state=42)

    # Leet the tweets
    df_to_leet[column_to_leet] = df_to_leet[column_to_leet].progress_apply(leet_data_augmenter,  augmenter=augmenter)
    df_to_leet["Camouflaged"] = True

    # count nan values in df_train_offen_to_leet["tweet"] column
    print(f"Nan values in '{column_to_leet}' column: ", df_to_leet[column_to_leet].isna().sum())

    # Substitute the original rows by the leeted version
    cols = list(df_leeted.columns) 
    df_leeted.loc[df_leeted.index.isin(df_to_leet.index), cols] = df_to_leet[cols]    

    display( df_leeted.groupby("Camouflaged").count() ) 
    
    return df_leeted



# Define la función random_augmenter
def random_augly_augmenter(input_text):
    # define augmenters
    aug_ReplaceSimilar = textaugs.ReplaceSimilarChars(p=0.4)
    aug_ReplaceSimUnicode = textaugs.ReplaceSimilarUnicodeChars(p=0.4)
    aug_InsertPunctuation = textaugs.InsertPunctuationChars(granularity="all", cadence=4, p=0.4)

    aug_ReplaceSim_Punct = textaugs.Compose(
        transforms=[
            textaugs.ReplaceSimilarChars(p=0.4),
            textaugs.InsertPunctuationChars(granularity="all", cadence=4, p=1)
        ],
        p=0.4
    )

    aug_ReplaceSimUnicode_Punct = textaugs.Compose(
        transforms=[
            textaugs.ReplaceSimilarUnicodeChars(
                p=1,
            ),
            textaugs.InsertPunctuationChars(granularity="all", cadence=4, p=0.4)
        ],
        p=1,
    )

    augmenters = [
        aug_ReplaceSimilar,
        aug_ReplaceSimUnicode,
        aug_InsertPunctuation,
        aug_ReplaceSim_Punct,
        aug_ReplaceSimUnicode_Punct,
    ]

    random_augmenter = rng.choice(
        augmenters, replace=True, 
        # p=[0.5, 0.2, 0.12, 0.12, 0.06]
    )
    leet_text = random_augmenter(input_text)
    
    if isinstance(leet_text, list):
        # Convierte la lista en una cadena usando un separador (por ejemplo, un espacio)
        leet_text = " ".join(leet_text)
    
    return leet_text

def create_augly_augmentation(df_ori, frac, augmenter, column_to_leet):
    # Create a copy of the original dataframe for saving the leeted version
    df_leeted = copy(df_ori)
    df_leeted["Camouflaged"] = False
    
    # Extract fraction to leet
    df_to_leet = df_leeted.sample(frac=frac, random_state=42)

    # Leet the tweets
    df_to_leet[column_to_leet] = df_to_leet[column_to_leet].progress_apply(augmenter)
    df_to_leet["Camouflaged"] = True

    # count nan values in df_train_offen_to_leet["tweet"] column
    print(f"Nan values in '{column_to_leet}' column: ", df_to_leet[column_to_leet].isna().sum())

    # Substitute the original rows by the leeted version
    cols = list(df_leeted.columns) 
    df_leeted.loc[df_leeted.index.isin(df_to_leet.index), cols] = df_to_leet[cols]    

    display( df_leeted.groupby("Camouflaged").count() )
    
    return df_leeted

# [OffenSenEval 2019](https://competitions.codalab.org/competitions/20011)

Sub-task A: Offensive or not

In this sub-task we are interested in the identification of offensive posts and posts containing any form of (untargeted) profanity. In this sub-task there are **2 categories** in which the tweet could be classified -

- **Not Offensive** - This post does not contain offense or profanity.  Non-offensive posts do not include any form of offense or profanity.

- **Offensive** - This post contains offensive language or a targeted (veiled or direct) offense.  In our annotation, we label a post as offensive if it  contains any form of non-acceptable language (profanity) or a targeted offense which can be veiled or direct. To sum up this category includes insults, threats, and posts containing profane language and swear words.

### Load Pandas data

Load the orignal non-camouflage data and transform it to spacy data format for Naive models training.

In [10]:
# Load Temporal Train
# train_offen_path = Path(home).joinpath("data/WordCamouflage_Resiliance/Datasets/OffensEval_2019/OffensEval_2019/OLIDv1.0/olid-training-v1.0.tsv")
train_offen_path = Path(home).joinpath("work/Code/WordCamouflage_Resiliance/code/tmp_data/Offen/olid-training-v1.0.tsv")

df_train_tmp_offen = pd.read_csv( train_offen_path, sep = "\t").drop(labels = ["id", "subtask_b", "subtask_c"], axis = 1)

# drop duplicates
df_train_tmp_offen = df_train_tmp_offen.drop_duplicates(subset = ["tweet"])

labels_offen = list(df_train_tmp_offen["subtask_a"].unique())

# Split into Train and Dev
df_train_offen, df_dev_offen = train_test_split(
    df_train_tmp_offen,
    random_state=42,
    test_size=0.1,
    stratify=df_train_tmp_offen["subtask_a"].to_numpy(),
)


# Load Test Data and Label
test_data_offen_path = Path(home).joinpath("work/Code/WordCamouflage_Resiliance/code/tmp_data/Offen/testset-levela.tsv")
test_label_offen_path = Path(home).joinpath("work/Code/WordCamouflage_Resiliance/code/tmp_data/Offen/labels-levela.csv")

df_test_data_offen = pd.read_csv( test_data_offen_path, sep = "\t")
df_test_label_offen = pd.read_csv( test_label_offen_path, sep = ",", header=None, names=["id", "test_label"])

# Merge Test Data and Test Label
df_test_offen = pd.merge(df_test_data_offen, df_test_label_offen, on="id", how="outer").drop(labels = ["id"], axis = 1)

print("Train")
display(df_train_offen.groupby("subtask_a").count())

print("Dev")
display(df_dev_offen.groupby("subtask_a").count())

print("Test")
display(df_test_offen.groupby("test_label").count())

Train


,tweet
subtask_a,
NOT,7933
OFF,3953


Dev


,tweet
subtask_a,
NOT,882
OFF,439


Test


,tweet
test_label,
NOT,620
OFF,240


#### From Pandas to Spacy

In [ ]:
# Train spacy, Dev spacy, Test spacy
train_offen_output_path = "./train_offen_toy.spacy"
dev_offen_output_path = "./dev_offen_toy.spacy"
test_offen_output_path = "./test_offen_toy.spacy"

# tuple of tuples. Each nested tuple is (Tweet, Label)
train_offen_data_tuples = tuple(df_train_offen.itertuples(index=False, name=None))
dev_offen_data_tuples = tuple(df_dev_offen.itertuples(index=False, name=None))
test_offen_data_tuples = tuple(df_test_offen.itertuples(index=False, name=None))

# Spact empty model
nlp = spacy.blank("en")

# Make spacy DocBin
train_offen_docs = make_docs(train_offen_data_tuples, nlp)
dev_offen_docs = make_docs(dev_offen_data_tuples, nlp)
test_offen_docs = make_docs(test_offen_data_tuples, nlp)

# save to binary file 
train_offen_doc_bin = DocBin(docs=train_offen_docs)
train_offen_doc_bin.to_disk(train_offen_output_path)
print(f"Processed Train {len(train_offen_data_tuples)} documents: {train_offen_output_path}")

dev_offen_doc_bin = DocBin(docs=dev_offen_docs)
dev_offen_doc_bin.to_disk(dev_offen_output_path)
print(f"Processed Dev {len(dev_offen_data_tuples)} documents: {dev_offen_output_path}")

test_offen_doc_bin = DocBin(docs=test_offen_docs)
test_offen_doc_bin.to_disk(test_offen_output_path)
print(f"Processed Test {len(test_offen_data_tuples)} documents: {test_offen_output_path}")

### Camouflage Data

In the following cells we generate the data for model fitting with the _static_ strategy for the different levels of complecity, word camouflage ratio and instance camouflage ratio. 

#### Level 1.1

In [ ]:
resiliance_level = "Level_1.1"
resiliance_easy = ["basic_leetspeak"]
augmenter = WordCamouflage_Augmenter.augmenter(
        extractor_type="yake",
        max_top_n=5,
        leet_punt_prb=0.9,
        leet_change_prb=0.8,
        leet_change_frq=0.8,
        leet_uniform_change=0.5,
        method=resiliance_easy,
)


########### 10% ############
df_train_offen_10_per = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_10_per = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_10_per= create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 100% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_10_per,
df_dev = df_dev_offen_10_per, 
df_test = df_test_offen_10_per, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/test.spacy",
lang="en",
labels=labels_offen
)

########### 25% ############
df_train_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 25% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_25_per_advanced_v2,
df_dev = df_dev_offen_25_per_advanced_v2, 
df_test = df_test_offen_25_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/test.spacy",
lang="en",
labels=labels_offen
)


########### 50% ############
df_train_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 50% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_50_per_advanced_v2,
df_dev = df_dev_offen_50_per_advanced_v2, 
df_test = df_test_offen_50_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/test.spacy",
lang="en",
labels=labels_offen
)

########### 75% ############
df_train_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 75% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_75_per_advanced_v2,
df_dev = df_dev_offen_75_per_advanced_v2, 
df_test = df_test_offen_75_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/test.spacy",
lang="en",
labels=labels_offen
)


########### 100% ############
df_train_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 100% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_100_per_advanced_v2,
df_dev = df_dev_offen_100_per_advanced_v2, 
df_test = df_test_offen_100_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/test.spacy",
lang="en",
labels=labels_offen
)

#### Level 1.2

In [ ]:
resiliance_level = "Level_1.2"
resiliance_easy = ["basic_leetspeak"]
augmenter = WordCamouflage_Augmenter.augmenter(
        extractor_type="yake",
        max_top_n=20,
        leet_punt_prb=0.9,
        leet_change_prb=0.8,
        leet_change_frq=0.8,
        leet_uniform_change=0.5,
        method=resiliance_easy,
)


########### 10% ############
df_train_offen_10_per = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_10_per = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_10_per= create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 100% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_10_per,
df_dev = df_dev_offen_10_per, 
df_test = df_test_offen_10_per, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/test.spacy",
lang="en",
labels=labels_offen
)

########### 25% ############
df_train_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 25% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_25_per_advanced_v2,
df_dev = df_dev_offen_25_per_advanced_v2, 
df_test = df_test_offen_25_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/test.spacy",
lang="en",
labels=labels_offen
)


########### 50% ############
df_train_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 50% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_50_per_advanced_v2,
df_dev = df_dev_offen_50_per_advanced_v2, 
df_test = df_test_offen_50_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/test.spacy",
lang="en",
labels=labels_offen
)

########### 75% ############
df_train_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 75% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_75_per_advanced_v2,
df_dev = df_dev_offen_75_per_advanced_v2, 
df_test = df_test_offen_75_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/test.spacy",
lang="en",
labels=labels_offen
)


########### 100% ############
df_train_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 100% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_100_per_advanced_v2,
df_dev = df_dev_offen_100_per_advanced_v2, 
df_test = df_test_offen_100_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/test.spacy",
lang="en",
labels=labels_offen
)

#### Level 2.1

In [ ]:
resiliance_level = "Level_2.1"
resiliance_intermediate = ["intermediate_leetspeak", "punct_camo"]
augmenter = WordCamouflage_Augmenter.augmenter(
        extractor_type="yake",
        max_top_n=5,
        leet_punt_prb=0.9,
        leet_change_prb=0.5,
        leet_change_frq=0.8,
        leet_uniform_change=0.6,
        punt_hyphenate_prb=0.7,
        punt_uniform_change_prb=0.95,
        punt_word_splitting_prb=0.8,
        method=resiliance_intermediate,
)


########### 10% ############
df_train_offen_10_per = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_10_per = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_10_per= create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 100% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_10_per,
df_dev = df_dev_offen_10_per, 
df_test = df_test_offen_10_per, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/test.spacy",
lang="en",
labels=labels_offen
)

########### 25% ############
df_train_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 25% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_25_per_advanced_v2,
df_dev = df_dev_offen_25_per_advanced_v2, 
df_test = df_test_offen_25_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/test.spacy",
lang="en",
labels=labels_offen
)


########### 50% ############
df_train_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 50% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_50_per_advanced_v2,
df_dev = df_dev_offen_50_per_advanced_v2, 
df_test = df_test_offen_50_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/test.spacy",
lang="en",
labels=labels_offen
)

########### 75% ############
df_train_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 75% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_75_per_advanced_v2,
df_dev = df_dev_offen_75_per_advanced_v2, 
df_test = df_test_offen_75_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/test.spacy",
lang="en",
labels=labels_offen
)


########### 100% ############
df_train_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 100% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_100_per_advanced_v2,
df_dev = df_dev_offen_100_per_advanced_v2, 
df_test = df_test_offen_100_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/test.spacy",
lang="en",
labels=labels_offen
)

#### Level 2.2

In [ ]:
resiliance_level = "Level_2.2"
resiliance_intermediate = ["intermediate_leetspeak", "punct_camo"]
augmenter = WordCamouflage_Augmenter.augmenter(
        extractor_type="yake",
        max_top_n=20,
        leet_punt_prb=0.9,
        leet_change_prb=0.5,
        leet_change_frq=0.8,
        leet_uniform_change=0.6,
        punt_hyphenate_prb=0.7,
        punt_uniform_change_prb=0.95,
        punt_word_splitting_prb=0.8,
        method=resiliance_intermediate,
)


########### 10% ############
df_train_offen_10_per = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_10_per = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_10_per= create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 100% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_10_per,
df_dev = df_dev_offen_10_per, 
df_test = df_test_offen_10_per, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/test.spacy",
lang="en",
labels=labels_offen
)

########### 25% ############
df_train_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 25% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_25_per_advanced_v2,
df_dev = df_dev_offen_25_per_advanced_v2, 
df_test = df_test_offen_25_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/test.spacy",
lang="en",
labels=labels_offen
)


########### 50% ############
df_train_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 50% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_50_per_advanced_v2,
df_dev = df_dev_offen_50_per_advanced_v2, 
df_test = df_test_offen_50_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/test.spacy",
lang="en",
labels=labels_offen
)

########### 75% ############
df_train_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 75% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_75_per_advanced_v2,
df_dev = df_dev_offen_75_per_advanced_v2, 
df_test = df_test_offen_75_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/test.spacy",
lang="en",
labels=labels_offen
)


########### 100% ############
df_train_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 100% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_100_per_advanced_v2,
df_dev = df_dev_offen_100_per_advanced_v2, 
df_test = df_test_offen_100_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/test.spacy",
lang="en",
labels=labels_offen
)

#### Level 3.1

In [ ]:
resiliance_level = "Level_3.1"
resiliance_advanced = ["advanced_leetspeak", "punct_camo", "inv_camo"]

augmenter = WordCamouflage_Augmenter.augmenter(
        extractor_type="yake",
        max_top_n=5,
        leet_punt_prb=0.4,
        leet_change_prb=0.5,
        leet_change_frq=0.8,
        leet_uniform_change=0.6,
        punt_hyphenate_prb=0.7,
        punt_uniform_change_prb=0.95,
        punt_word_splitting_prb=0.8,
        inv_max_dist=4,
        inv_only_max_dist_prb=0.5,
        method=resiliance_advanced,
)


########### 10% ############
df_train_offen_10_per = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_10_per = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_10_per= create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 100% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_10_per,
df_dev = df_dev_offen_10_per, 
df_test = df_test_offen_10_per, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/test.spacy",
lang="en",
labels=labels_offen
)

########### 25% ############
df_train_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 25% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_25_per_advanced_v2,
df_dev = df_dev_offen_25_per_advanced_v2, 
df_test = df_test_offen_25_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/test.spacy",
lang="en",
labels=labels_offen
)


########### 50% ############
df_train_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 50% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_50_per_advanced_v2,
df_dev = df_dev_offen_50_per_advanced_v2, 
df_test = df_test_offen_50_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/test.spacy",
lang="en",
labels=labels_offen
)

########### 75% ############
df_train_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 75% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_75_per_advanced_v2,
df_dev = df_dev_offen_75_per_advanced_v2, 
df_test = df_test_offen_75_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/test.spacy",
lang="en",
labels=labels_offen
)


########### 100% ############
df_train_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 100% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_100_per_advanced_v2,
df_dev = df_dev_offen_100_per_advanced_v2, 
df_test = df_test_offen_100_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/test.spacy",
lang="en",
labels=labels_offen
)

#### Level 3.2

In [ ]:
resiliance_level = "Level_3.2"
resiliance_advanced = ["advanced_leetspeak", "punct_camo", "inv_camo"]

augmenter = WordCamouflage_Augmenter.augmenter(
        extractor_type="yake",
        max_top_n=20,
        leet_punt_prb=0.4,
        leet_change_prb=0.5,
        leet_change_frq=0.8,
        leet_uniform_change=0.6,
        punt_hyphenate_prb=0.7,
        punt_uniform_change_prb=0.95,
        punt_word_splitting_prb=0.8,
        inv_max_dist=4,
        inv_only_max_dist_prb=0.5,
        method=resiliance_advanced,
)


########### 10% ############
df_train_offen_10_per = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_10_per = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_10_per= create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 100% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_10_per,
df_dev = df_dev_offen_10_per, 
df_test = df_test_offen_10_per, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/test.spacy",
lang="en",
labels=labels_offen
)

########### 25% ############
df_train_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 25% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_25_per_advanced_v2,
df_dev = df_dev_offen_25_per_advanced_v2, 
df_test = df_test_offen_25_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/test.spacy",
lang="en",
labels=labels_offen
)


########### 50% ############
df_train_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 50% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_50_per_advanced_v2,
df_dev = df_dev_offen_50_per_advanced_v2, 
df_test = df_test_offen_50_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/test.spacy",
lang="en",
labels=labels_offen
)

########### 75% ############
df_train_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 75% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_75_per_advanced_v2,
df_dev = df_dev_offen_75_per_advanced_v2, 
df_test = df_test_offen_75_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/test.spacy",
lang="en",
labels=labels_offen
)


########### 100% ############
df_train_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 100% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_100_per_advanced_v2,
df_dev = df_dev_offen_100_per_advanced_v2, 
df_test = df_test_offen_100_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/test.spacy",
lang="en",
labels=labels_offen
)

#### Level Mixed

Mixed camouflaged version data combining the different levels of complexity.

In [ ]:
resiliance_level = "Level_Mixed"
augmenter = WordCamouflage_Augmenter.augmenter(
        extractor_type="yake",
        max_top_n=20,
        leet_punt_prb=0.4,
        leet_change_prb=0.5,
        leet_change_frq=0.8,
        leet_uniform_change=0.6,
        punt_hyphenate_prb=0.7,
        punt_uniform_change_prb=0.95,
        punt_word_splitting_prb=0.8,
        inv_max_dist=4,
        inv_only_max_dist_prb=0.5
)


########### 10% ############
df_train_offen_10_per = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_10_per = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_10_per= create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.1, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 100% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_10_per,
df_dev = df_dev_offen_10_per, 
df_test = df_test_offen_10_per, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/test.spacy",
lang="en",
labels=labels_offen
)

########### 25% ############
df_train_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_25_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.25, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 25% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_25_per_advanced_v2,
df_dev = df_dev_offen_25_per_advanced_v2, 
df_test = df_test_offen_25_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/test.spacy",
lang="en",
labels=labels_offen
)


########### 50% ############
df_train_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_50_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.5, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 50% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_50_per_advanced_v2,
df_dev = df_dev_offen_50_per_advanced_v2, 
df_test = df_test_offen_50_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/test.spacy",
lang="en",
labels=labels_offen
)

########### 75% ############
df_train_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_75_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=0.75, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 75% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_75_per_advanced_v2,
df_dev = df_dev_offen_75_per_advanced_v2, 
df_test = df_test_offen_75_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/test.spacy",
lang="en",
labels=labels_offen
)


########### 100% ############
df_train_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_train_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)
df_dev_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_dev_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)
df_test_offen_100_per_advanced_v2 = create_leet_augmenter_df(
    df_ori=df_test_offen, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)

# Save the 100% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_100_per_advanced_v2,
df_dev = df_dev_offen_100_per_advanced_v2, 
df_test = df_test_offen_100_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/test.spacy",
lang="en",
labels=labels_offen
)

### TRAINING

#### Dynamic

```bash
python -m spacy train  [config-path] --output [output-path] --gpu-id X --code [augmenter-script]
```

For example, training the BERT encoder-model in dynamic strategy with 10% of data instances camouflaged with mixed camouflaged. 
```bash
python -m spacy train ~work/NLP-Camouflage-is-all-you-need/Experiments/Fine_tuning_robustness/configs/offen_semeval_10_dynamic_config.cfg --output ~work/NLP-Camouflage-is-all-you-need/output_models/bert-base-uncased_offen-10-dynamic --gpu-id 0 --code functions.py
```

#### Static

In this case, the `config-path` track a config file where the data used is the one previously camouflaged

```bash
python -m spacy train  [config-path] --output [output-path] --gpu-id X 
```

For example, training the BERT encoder-model in static strategy with 10% of data instances camouflaged with mixed camouflaged. 
```bash
python -m spacy train ~work/NLP-Camouflage-is-all-you-need/Experiments/Fine_tuning_robustness/configs/offen_semeval_10_static_config.cfg --output ~work/NLP-Camouflage-is-all-you-need/output_models/bert-base-uncased_offen-10-static --gpu-id 0
```

### Evaluate

#### Dynamic

10%, Level 1 v2

```bash
python -m spacy evaluate [model-to-path] [test-data-path] --gpu-id X --output [output-results-path]
```

```bash
python -m spacy evaluate ~work/NLP-Camouflage-is-all-you-need/output_models/bert-base-uncased_offen-10-dynamic ~work/NLP-Camouflage-is-all-you-need/Spacy_Data/Offen_SemEval_2019/Leet_Data/Level_1.2/10_per/test.spacy --gpu-id 1 --output ~work/NLP-Camouflage-is-all-you-need/output_models/bert-base-uncased_offen-10-dynamic/model-best/test_10_level_1.2_result.json
```

#### Static

10%, Level 1 v2

```bash
python -m spacy evaluate [model-to-path] [test-data-path] --gpu-id X --output [output-results-path]
```

```bash
python -m spacy evaluate ~work/NLP-Camouflage-is-all-you-need/output_models/bert-base-uncased_offen-10-static ~work/NLP-Camouflage-is-all-you-need/Spacy_Data/Offen_SemEval_2019/Leet_Data/Level_1.2/10_per/test.spacy --gpu-id 1 --output ~work/NLP-Camouflage-is-all-you-need/output_models/bert-base-uncased_offen-10-static/model-best/test_10_level_1.2_result.json
```

### AugLy

External validation with AugLy augmentation

In [42]:
import augly.text as textaugs
import numpy as np

# Crea un generador con la semilla 42
rng = np.random.default_rng(42)

In [10]:
resiliance_level = "AugLy"

########### 10% ############
df_train_offen_10_per = create_augly_augmentation(
    df_ori=df_train_offen, frac=0.1, augmenter=random_augly_augmenter, column_to_leet = "tweet"
)
df_dev_offen_10_per = create_augly_augmentation(
    df_ori=df_dev_offen, frac=0.1, augmenter=random_augly_augmenter, column_to_leet = "tweet"
)
df_test_offen_10_per= create_augly_augmentation(
    df_ori=df_test_offen, frac=0.1, augmenter=random_augly_augmenter, column_to_leet = "tweet"
)

# Save the 100% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_10_per,
df_dev = df_dev_offen_10_per, 
df_test = df_test_offen_10_per, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/10_per/test.spacy",
lang="en",
labels=labels_offen
)

########### 25% ############
df_train_offen_25_per_advanced_v2 = create_augly_augmentation(
    df_ori=df_train_offen, frac=0.25, augmenter=random_augly_augmenter, column_to_leet = "tweet"
)
df_dev_offen_25_per_advanced_v2 = create_augly_augmentation(
    df_ori=df_dev_offen, frac=0.25, augmenter=random_augly_augmenter, column_to_leet = "tweet"
)
df_test_offen_25_per_advanced_v2 = create_augly_augmentation(
    df_ori=df_test_offen, frac=0.25, augmenter=random_augly_augmenter, column_to_leet = "tweet"
)

# Save the 25% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_25_per_advanced_v2,
df_dev = df_dev_offen_25_per_advanced_v2, 
df_test = df_test_offen_25_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/25_per/test.spacy",
lang="en",
labels=labels_offen
)


########### 50% ############
df_train_offen_50_per_advanced_v2 = create_augly_augmentation(
    df_ori=df_train_offen, frac=0.5, augmenter=random_augly_augmenter, column_to_leet = "tweet"
)
df_dev_offen_50_per_advanced_v2 = create_augly_augmentation(
    df_ori=df_dev_offen, frac=0.5, augmenter=random_augly_augmenter, column_to_leet = "tweet"
)
df_test_offen_50_per_advanced_v2 = create_augly_augmentation(
    df_ori=df_test_offen, frac=0.5, augmenter=random_augly_augmenter, column_to_leet = "tweet"
)

# Save the 50% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_50_per_advanced_v2,
df_dev = df_dev_offen_50_per_advanced_v2, 
df_test = df_test_offen_50_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/50_per/test.spacy",
lang="en",
labels=labels_offen
)

########### 75% ############
df_train_offen_75_per_advanced_v2 = create_augly_augmentation(
    df_ori=df_train_offen, frac=0.75, augmenter=random_augly_augmenter, column_to_leet = "tweet"
)
df_dev_offen_75_per_advanced_v2 = create_augly_augmentation(
    df_ori=df_dev_offen, frac=0.75, augmenter=random_augly_augmenter, column_to_leet = "tweet"
)
df_test_offen_75_per_advanced_v2 = create_augly_augmentation(
    df_ori=df_test_offen, frac=0.75, augmenter=random_augly_augmenter, column_to_leet = "tweet"
)

# Save the 75% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_75_per_advanced_v2,
df_dev = df_dev_offen_75_per_advanced_v2, 
df_test = df_test_offen_75_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/75_per/test.spacy",
lang="en",
labels=labels_offen
)


########### 100% ############
df_train_offen_100_per_advanced_v2 = create_augly_augmentation(
    df_ori=df_train_offen, frac=1, augmenter=random_augly_augmenter, column_to_leet = "tweet"
)
df_dev_offen_100_per_advanced_v2 = create_augly_augmentation(
    df_ori=df_dev_offen, frac=1, augmenter=random_augly_augmenter, column_to_leet = "tweet"
)
df_test_offen_100_per_advanced_v2 = create_augly_augmentation(
    df_ori=df_test_offen, frac=1, augmenter=random_augly_augmenter, column_to_leet = "tweet"
)

# Save the 100% leeted difficult dataframes to spacy binary files
pd_2_spacy(df_train = df_train_offen_100_per_advanced_v2,
df_dev = df_dev_offen_100_per_advanced_v2, 
df_test = df_test_offen_100_per_advanced_v2, 
train_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/train.spacy", 
dev_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/dev.spacy",
test_output_path = f"./Spacy_Data/Offen_SemEval_2019/Leet_Data/{resiliance_level}/100_per/test.spacy",
lang="en",
labels=labels_offen
)

100% 1189/1189 [00:01<00:00, 853.93it/s]

Nan values in 'tweet' column:  0


,tweet,subtask_a
Camouflaged,,
False,10697,10697
True,1189,1189


100% 132/132 [00:00<00:00, 895.62it/s]


Nan values in 'tweet' column:  0


...

Processed Train 11886 documents: ./Spacy_Data/Offen_SemEval_2019/Leet_Data/AugLy/100_per/train.spacy


Making docs: 100% 1321/1321 [00:00<00:00, 1744.45it/s]


Processed Dev 1321 documents: ./Spacy_Data/Offen_SemEval_2019/Leet_Data/AugLy/100_per/dev.spacy


Making docs: 100% 860/860 [00:00<00:00, 1213.29it/s]


Processed Test 860 documents: ./Spacy_Data/Offen_SemEval_2019/Leet_Data/AugLy/100_per/test.spacy


##### Evaluation of the models on AugLy

```bash
python -m spacy evaluate [path-to-model] [path-to-augly-data] --gpu-id X --output [output-path]
```

For example for BERT (encoder-only model):
```bash
python -m spacy evaluate ~/work/Code/WordCamouflage_Resiliance/code/output_models/Offen_SemEval_2019/bert-base-uncased_naive/model-best ~/work/Code/WordCamouflage_Resiliance/code/Spacy_Data/Offen_SemEval_2019/Leet_Data/AugLy/10_per/test.spacy --gpu-id 0 --output ~/work/Code/WordCamouflage_Resiliance/code/output_models/Offen_SemEval_2019/bert-base-uncased_naive/model-best/test_augly_10_result.json
```

In [71]:
df_test_offen.iloc[30]

tweet         #ChristineBlaseyFord is your #Kavanaugh accuser... #Liberals try this EVERY time... #ConfirmJudgeKavanaugh URL
test_label                                                                                                               OFF
Name: 30, dtype: object

In [24]:
aug_ReplaceSimilar = textaugs.ReplaceSimilarChars(p=1)
aug_InsertPunctuation = textaugs.InsertPunctuationChars(granularity="word")

In [25]:
aug_InsertPunctuation("HElloo world")

['H!E!l!l!o!o w.o.r.l.d']

In [32]:
pd.set_option("display.max_colwidth", None)
df_test_offen_10_per[df_test_offen_10_per["Camouflaged"] == True]

,tweet,test_label,Camouflaged
23,#SilsilaBadallteRishtonKa テag is filled with appreciation o₣ @ USER only.... Never watch£Đ this serial but I always wantⒺd appreciation for Adiťi... She iŜ underrated sᎥnce sta℟ⓣing of the shoŵ... She Ïs wỢrth Ớf aľl this appᖇeciatĬoŋ.. 👏 👏 👏,NOT,True
30,#ChristineBlaseyFord is your#K@vanau9h accuser...#L7|3erals t|2y +his EVERY tim3...#ConfirmJudgeKavanaugh URL,OFF,True
39,@ USER How is she hiding her ugly perso^ali+y. S|-|e is t|-|e worst.,OFF,True
63,@USE:R om:g is: he :for :real: ?!?:!!??:????:) th:is h:appe:ned :in p:eru :like: 40 :year:s ag:o an:d th:e In:ti d:eval:uate:d so: fuc:king: muc:h th:at t:hey :had :to c:omol:etel:y ch:ange: the: coi:n sy:stem: bec:ause: our: mon:ey w:as w:orth:less: i-,OFF,True
65,#Halloween Sign If You Are Reάᚦing Thi₰#CreepingUp Behind ௶ou Ꮗood Wall ᗫeċôration#tmŤinstã#cwsigns URL via @ USER,NOT,True
...,...,...,...
836,"#Antifa invaded a ጠemoriaȴ in recognition of Katę Steinle in AuŞtin, Texas, a₦d tried ṫo comfort hiᗰ it loỞked like when it wÀs struggling, but ℍas ℝeturnÊd to the Southern District Óf NÉw York, is r∑cuŝed froᛖ tℏe Michaҿł Cohen raid, Trump has my full support. Moving on.",NOT,True
846,"I'am sÔ sleepy.. {Boテ cüddle§ up to yoน} {#RoguẾBoŧ} """,NOT,True
850,"1) WoVV, saf3 sex! 7hat's hot! We lov3 a butch top who @lso receives! URL",NOT,True
856,#MeetTℍeŚpeaⓀeŗs 🙌 @ UՏER will ⓟreseӇt in όur event OIฟ 2018: Finpact-Global Impact through FinÅⓝcial Technologies. She is Senior Advisor GroŲp Sustainable ḞinaŇce and worked on green e₦ergy Ẵnd Ĉlimaté ri$k. Join us to mҾet Thina URL#oiw2018 URL,NOT,True


### Table Paper

Random examples extracted from the camouflaged data across different levels of complexities

In [58]:
#### 10% Leet ####
toy_test = df_test_offen.sample(15, random_state=2)

# resiliance_level = "Level_3"
# resiliance_intermediate = ["intermediate_leetspeak", "punct_camo"]

# augmenter = WordCamouflage_Augmenter.augmenter(
#         extractor_type="yake",
#         max_top_n=20,
#         leet_punt_prb=0.9,
#         leet_change_prb=0.5,
#         leet_change_frq=0.8,
#         leet_uniform_change=0.6,
#         punt_hyphenate_prb=0.7,
#         punt_uniform_change_prb=0.95,
#         punt_word_splitting_prb=0.8,
#         method=resiliance_intermediate,
# )

resiliance_level = "Level_1.1"
resiliance_easy = ["basic_leetspeak"]
augmenter = WordCamouflage_Augmenter.augmenter(
        extractor_type="yake",
        max_top_n=5,
        leet_punt_prb=0.9,
        leet_change_prb=0.8,
        leet_change_frq=0.8,
        leet_uniform_change=0.5,
        method=resiliance_easy,
)

# Create a test dataframe with 10% of leeted tweets
df_level_1 = create_leet_augmenter_df(
    df_ori=toy_test, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)

resiliance_level = "Level_2.1"
resiliance_intermediate = ["intermediate_leetspeak", "punct_camo"]
augmenter = WordCamouflage_Augmenter.augmenter(
        extractor_type="yake",
        max_top_n=5,
        leet_punt_prb=0.9,
        leet_change_prb=0.5,
        leet_change_frq=0.8,
        leet_uniform_change=0.6,
        punt_hyphenate_prb=0.7,
        punt_uniform_change_prb=0.95,
        punt_word_splitting_prb=0.8,
        method=resiliance_intermediate,
)

df_level_2 = create_leet_augmenter_df(
    df_ori=toy_test, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)

resiliance_level = "Level_3.1"
resiliance_advanced = ["advanced_leetspeak", "punct_camo", "inv_camo"]

augmenter = WordCamouflage_Augmenter.augmenter(
        extractor_type="yake",
        max_top_n=5,
        leet_punt_prb=0.4,
        leet_change_prb=0.5,
        leet_change_frq=0.8,
        leet_uniform_change=0.6,
        punt_hyphenate_prb=0.7,
        punt_uniform_change_prb=0.95,
        punt_word_splitting_prb=0.8,
        inv_max_dist=4,
        inv_only_max_dist_prb=0.5,
        method=resiliance_advanced,
)

df_level_3 = create_leet_augmenter_df(
    df_ori=toy_test, frac=1, augmenter=augmenter, column_to_leet = "tweet"
)

resiliance_level = "AugLy"

########### 10% ############
df_augly = create_augly_augmentation(
    df_ori=toy_test, frac=1, augmenter=random_augly_augmenter, column_to_leet = "tweet"
)
# toy_test = df_test_offen.sample(15, random_state=42)
pd.set_option('max_colwidth', -1)
display(toy_test.head())
print()
print()
print("------------- LEVEL 1 --------------")
display(df_level_1.head(5))

print()
print()
print("------------- LEVEL 2 --------------")
display(df_level_2.head(5))

print()
print()
print("------------- LEVEL 3 --------------")
display(df_level_3.head(5))

print()
print()
print("------------- AugLy --------------")
display(df_augly.head(5))


100% 15/15 [00:00<00:00, 111.94it/s]

Nan values in 'tweet' column:  0


,tweet,test_label
Camouflaged,,
True,15,15


100% 15/15 [00:00<00:00, 70.43it/s]

Nan values in 'tweet' column:  0


...

,tweet,test_label,Camouflaged
286,@'ū'S'E'R @(_)ŝEŗ @UşEř ĝū₦ Cꬽn†`r0∟ is nothing but )|)>)e)ŏ)|)>)∟)e /C/o/ͷ/₸/r/o/ļ. The left wants Gũñ CØntЯʘ| ONLY because they do NOT want us little guys armed &amp; able to defend ourselves. Makes it harder to get us behind barbed-wire fences as they saw in North |<oЯeă.,NOT,True
169,"#D1śɅŗ]V[₸HE]V[ #A|\|TıFA Gıv℮ ]ą©k here a participation trophy! ""U""®""L",NOT,True
363,"6 äNţI<Fä ATTACKED ĉOP$, |4|ŗ|ŗ|E|ϟ|7|E|D AT RALLY IN DEηṿE#R, MED!A BLACKOUT URL",NOT,True
258,"#Cin/cin\naŧi;En^quiℝeℝ steps into #ģũ{\} =C=ŏ=ŋ=t=ŗ=ŏ=ℓ FINäłłꭚ. Took a #MassShhoting on their damn door step for their #editors to wake up, but better late than never. UR∟",OFF,True
74,@U.5.ƹR Well she is }č}h}Ʊ}č}k ś©ĥŬmer's cousin or niece or something so she is obviously part of the Dŉ[ conspiracy against !N!ĭ!ꭙ!o!n. Or part of the Deep State. Or a Zionist plot. Or 'Big comedy'. Or whatever the hell today's conspiracy theory is.,NOT,True




------------- LEVEL 3 --------------


,tweet,test_label,Camouflaged
286,@UϟEℝ @µSœR @]ų]S]E]Ɽ Gun ℂöπţröĺ is nothing but +p+ë+ꬽ+p+ℒ+℮ ç[]₦ţ®Ɵℒ. The left wants geeบņ C<>ทₜr<>l ONLY because they do NOT want us little guys armed &amp; able to defend ourselves. Makes it harder to get us behind barbed-wire fences as they saw in North Kȱrea.,NOT,True
169,#[)ISA/2₥₸HE₥ #*A*N*T*I*|*#*A Gĩve Jaç|< here a participation trophy! ŬRL,NOT,True
363,"6 TI FAAN ATTACKED CōPS, +A+R+R+E+S+†+E+[+) AT RALLY IN |)Eℕ|/Eŕ, MED|Â BLACKOUT URL",NOT,True
258,"#+ℂ+ï+ŉ+ℂ+ï+ŉ+ŉ+a+'+]+[+'+ï+E+ŉ+q+u+ï+𝔑+e+𝔑 steps into #Gun {ŏn}trʘl FINÃ|_|Y. Took a #MassShhoting on their damn door step for their #editors to wake up, but better late than never. ũRL",OFF,True
74,@.ʊ.∫.Æ.R Well she is Chücĸ \$\c\h\u\ℳ\ĕ\r's cousin or niece or something so she is obviously part of the ďN© conspiracy against /\/įχȱ/\/. Or part of the Deep State. Or a Zionist plot. Or 'Big comedy'. Or whatever the hell today's conspiracy theory is.,NOT,True




------------- AugLy --------------


,tweet,test_label,Camouflaged
286,@USE;R @U;SER ;@USE;R Gu;n Co;ntro;l is; not;hing; but; peo;ple ;cont;rol.; The; lef;t wa;nts ;gun ;cont;rol ;ONLY; bec;ause; the;y do; NOT; wan;t us; lit;tle ;guys; arm;ed &;amp;; abl;e to; def;end ;ours;elve;s. M;akes; it ;hard;er t;o ge;t us; beh;ind ;barb;ed-w;ire ;fenc;es a;s th;ey s;aw i;n No;rth ;Kore;a.,NOT,True
169,#DISARMTHEM #ANTIFA Give Jack here a participation trophy! URL,NOT,True
363,"6 ANTIFA ATTACKED COPS, ARRESTED AT RALLY IN DENVER, MEDIA BLACKOUT URL",NOT,True
258,"#Cίncinⓝa⍡iℯnqϋirer steps into#Gun Control FINALLY. Took a#MassShhoting on their damn door step foᶉ Ṫheir#editors tỞ wakҿ ửp, ხut better Ļate than Ŋever. URL",OFF,True
74,@USER Well she is Chuck Schumer's cousin or niece or something so she is obviously part of the DNC conspiracy against Nixon. Or part of the Deep State. Or a Zionist plot. Or 'Big comedy'. Or whatever the hell today's conspiracy theory is.,NOT,True


In [32]:
toy_test = df_test_offen.sample(15, random_state=2)
pd.set_option('max_colwidth', -1)
display(toy_test.head())

,tweet,test_label
286,@USER @USER @USER Gun Control is nothing but people control. The left wants gun control ONLY because they do NOT want us little guys armed &amp; able to defend ourselves. Makes it harder to get us behind barbed-wire fences as they saw in North Korea.,NOT
169,#DISARMTHEM #ANTIFA Give Jack here a participation trophy! URL,NOT
363,"6 ANTIFA ATTACKED COPS, ARRESTED AT RALLY IN DENVER, MEDIA BLACKOUT URL",NOT
258,"#CincinnatiEnquirer steps into #Gun Control FINALLY. Took a #MassShhoting on their damn door step for their #editors to wake up, but better late than never. URL",OFF
74,@USER Well she is Chuck Schumer's cousin or niece or something so she is obviously part of the DNC conspiracy against Nixon. Or part of the Deep State. Or a Zionist plot. Or 'Big comedy'. Or whatever the hell today's conspiracy theory is.,NOT


In [46]:
# toy_test = df_test_offen.sample(15, random_state=42)
pd.set_option('max_colwidth', -1)
display(toy_test.head())
print()
print()
print("------------- LEVEL 1 --------------")
display(df_level_1.head(5))

print()
print()
print("------------- LEVEL 2 --------------")
display(df_level_2.head(5))

print()
print()
print("------------- LEVEL 3 --------------")
display(df_level_3.head(5))

print()
print()
print("------------- AugLy --------------")
display(df_augly.head(5))


,tweet,test_label
286,@USER @USER @USER Gun Control is nothing but people control. The left wants gun control ONLY because they do NOT want us little guys armed &amp; able to defend ourselves. Makes it harder to get us behind barbed-wire fences as they saw in North Korea.,NOT
169,#DISARMTHEM #ANTIFA Give Jack here a participation trophy! URL,NOT
363,"6 ANTIFA ATTACKED COPS, ARRESTED AT RALLY IN DENVER, MEDIA BLACKOUT URL",NOT
258,"#CincinnatiEnquirer steps into #Gun Control FINALLY. Took a #MassShhoting on their damn door step for their #editors to wake up, but better late than never. URL",OFF
74,@USER Well she is Chuck Schumer's cousin or niece or something so she is obviously part of the DNC conspiracy against Nixon. Or part of the Deep State. Or a Zionist plot. Or 'Big comedy'. Or whatever the hell today's conspiracy theory is.,NOT




------------- LEVEL 1 --------------


,tweet,test_label,Camouflaged
286,@ƱS₤R @ųSER @ųSER G_n Cőntről is nothing but peØple Control. The lëft wants Gun Cōntrōl ONLY because they do NOT want us little gũys αrm₤d &amp; able to d3f3nd ourselves. Mäkës it hąrd3r to get us behind bȁrbed-wíre fꬲncės as they saw in N0rth KōrƏą.,NOT,True
169,#DISăRMTHēM #äNTĭFä Gĩvę Jäck here a p∆rtïc!p∆t!0n trŏphy! ŬRL,NOT,True
363,"6 ANT1FA ∀TT∀CKED CƟPS, ARRESTED AT RДLLY IN DƏNV3R, M€DI∀ BL@CKʘบT ũRL",NOT,True
258,"#C1nc1nnαt1ěnqบ1rěr stƸps into #Gün C<>ntr<>l FíNALLY. Took a #MДssShhƟtįng on their dɅmn dŏŏr stƐp for their #Œdįt<>rs to wākƏ up, but better lȁte than never. ŬRL",OFF,True
74,@ųSER Well she is Chųck Sch_mėr's cʘũs1n or nįÆcē or something so she is obviously pⱥrt of the DNC cʘnsp!r4cy against Nixon. Or part of the D€€p St∆tŒ. Or a Z¡<>n¡st plöt. Or 'Big c<>mÆdy'. Or whatever the hƐll tȱdⱥy's cȱnspírȃcy thƏőry is.,NOT,True




------------- LEVEL 2 --------------


,tweet,test_label,Camouflaged
286,"@#U#ş#ȇ#R @USĕ® @UśƹR ğ(_)|\| çoņႵrol is nothing but pe<>ple ,ć,ö,|,\,|,t,ŗ,ö,l. The left wants G(_)η çΘnₜЯʘ| ONLY because they do NOT want us little guys armed &amp; able to defend ourselves. Makes it harder to get us behind barbed-wire fences as they saw in North Koreȁ.",NOT,True
169,#DI5ARMᵗHœM #4ℕǂ1ϝ4 $ġ$ı$v$e Jac𝕂 here a participation trophy! URL,NOT,True
363,"6 :A:N:ͳ:I:ſ:A ATTACKED CØℙ§, äR`REš†EĎ AT RALLY IN |)Eň۷ER, M℮|)įA BLACKOUT URL",NOT,True
258,"#[¡n[¡n-năŧ¡En-qų¡ŗeŗ steps into #G(_)n ćonŢŗol F1ℕALLY. Took a #MassShhoting on their damn door step for their #editors to wake up, but better late than never. URł",OFF,True
74,@Ŭ$EЯ Well she is Cĥucķ #S#[#h#ų#m#e#r's cousin or niece or something so she is obviously part of the DNC conspiracy against Ni><on. Or part of the Deep State. Or a Zionist plot. Or 'Big comedy'. Or whatever the hell today's conspiracy theory is.,NOT,True




------------- LEVEL 3 --------------


,tweet,test_label,Camouflaged
286,"@USE|` @{└{┘{S{E{R @ũ$E2 ""G""u""|""\""| ""tŕo1Con is nothing but _peo_p_le /C/ŏ/n/t/r/ꬽ/|. The left wants {,un @C@[@]@n@t@r@ö@ļ ONLY because they do NOT want us little guys armed &amp; able to defend ourselves. Makes it harder to get us behind barbed-wire fences as they saw in North 1<orƐa.",NOT,True
169,#ĐıSÃRɱTⱨE₥ #∀N]TI]F∀ (+į√e Jac/< here a participation trophy! &ū&R&L,NOT,True
363,"6 ∆NŦI|F∆ ATTACKED COPş, AR^RēSȶēD AT RALLY IN DEη۷ER, DIA^^ě_ BLACKOUT URL",NOT,True
258,"#©||\|©||\||\|āt|E|\|qu|ℝeℝ steps into #~G~ʊ~[~\~] !ċ!o!n!Ⴕ!ŕ!o!£ }₣}ĭ}{}\}}}A}L}L}Y. Took a #MassShhoting on their damn door step for their #editors to wake up, but better late than never. üRł",OFF,True
74,@ŭ$ER Well she is ↄhu<Ķ šↄhu^^ȇr's cousin or niece or something so she is obviously part of the \D\N\C conspiracy against ń i χ φ ń. Or part of the Deep State. Or a Zionist plot. Or 'Big comedy'. Or whatever the hell today's conspiracy theory is.,NOT,True




------------- AugLy --------------


,tweet,test_label,Camouflaged
286,@ US3R @ USER @ USER Gun Control is ^oth|ng but p3ople control. T/-/e left wants gun control O^LY because they dD ^OT want us little 9uys armed & amp; able to |)efend ()urse|_ves. Makes it harder to get us behind barbed-wire fences as th3y saw i^ North Kor3a.,NOT,True
169,#DISARMTHEM#ANTIF4 Give Jack her3 a participation trophy! U/2L,NOT,True
363,"6 ANTIFA ATTACKED COPS, ARRESTED AT RALLY IN DENVER, MEDIA BLACKOUT URL",NOT,True
258,"#Cin!cinn!atiE!nqui!rer !step!s in!to #!Gun !Cont!rol !FINA!LLY.! Too!k a !#Mas!sShh!otin!g on! the!ir d!amn !door! ste!p fo!r th!eir !#edi!tors! to !wake! up,! but! bet!ter !late! tha!n ne!ver.! URL",OFF,True
74,@USE.R We.ll s.he i.s Ch.uck .Schu.mer'.s co.usin. or .niec.e or. som.ethi.ng s.o sh.e is. obv.ious.ly p.art .of t.he D.NC c.onsp.irac.y ag.ains.t Ni.xon.. Or .part. of .the .Deep. Sta.te. .Or a. Zio.nist. plo.t. O.r 'B.ig c.omed.y'. .Or w.hate.ver .the .hell. tod.ay's. con.spir.acy .theo.ry i.s.,NOT,True


### NAIVE



- [x] Naive
    - [x] test
    ```bash
    python -m spacy evaluate ~/work/Code/WordCamouflage_Resiliance/code/output_models/Constraint/bert-base-uncased_constraint-naive/model-best ~/work/Code/WordCamouflage_Resiliance/code/Spacy_Data/Constraint/Ori_Data/test.spacy  --gpu-id 1 --output ~/work/Code/WordCamouflage_Resiliance/code/output_models/Constraint/bert-base-uncased_constraint-naive/model-best/test_result.json
    
python -m spacy evaluate ~/work/Code/WordCamouflage_Resiliance/code/output_models/Constraint/bert-base-uncased_constraint-naive/model-best ~/work/Code/WordCamouflage_Resiliance/code/Spacy_Data/Constraint/Leet_Data/Level_3.2/100_per/test.spacy  --gpu-id 1 --output ~/work/Code/WordCamouflage_Resiliance/code/output_models/Constraint/bert-base-uncased_constraint-naive/model-best/test_100_level_3.2_result.json
    ```
    <br>
    
    - [x] 10
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_naive/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/10_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_naive/model-best/test_10_leet_result.json
    ```
    <br>
    
    - [x] 25
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_naive/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/25_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_naive/model-best/test_25_leet_result.json
    ```
    <br>
    - [x] 50
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_naive/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/50_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_naive/model-best/test_50_leet_result.json
    ```
    <br>
    - [x] 75
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_naive/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/75_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_naive/model-best/test_75_leet_result.json
    ```
    <br>
    - [x] 100
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_naive/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/100_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_naive/model-best/test_100_leet_result.json
    ```
    <br>

### 10_leet

- [x] 10_leet
    - [x] test
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Ori_Data/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_leet/model-best/test_result.json
    ```
    <br>
    - [x] 10
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/10_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_leet/model-best/test_10_leet_result.json
    ```
    <br>
    - [x] 25
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/25_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_leet/model-best/test_25_leet_result.json
    ```
    <br>    
    - [x] 50
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/50_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_leet/model-best/test_50_leet_result.json
    ```
    <br>
    - [x] 75
       ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/75_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_leet/model-best/test_75_leet_result.json
    ```
    <br>
    - [x] 100
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/100_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_leet/model-best/test_100_leet_result.json
    ```
    <br>

### 25 leet

- [ ] 25_leet
    - [x] test
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Ori_Data/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_leet/model-best/test_result.json
    ```
    <br>
    - [x] 10
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/10_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_leet/model-best/test_10_leet_result.json
    ```
    <br>
    - [x] 25
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/25_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_leet/model-best/test_25_leet_result.json
    ```
    <br>
    - [x] 50
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/50_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_leet/model-best/test_50_leet_result.json
    ```
    <br>
    - [x] 75
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/75_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_leet/model-best/test_75_leet_result.json
    ```
    <br>
    - [x] 100
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/100_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_leet/model-best/test_100_leet_result.json
    ```
    <br>

### 50 leet

- [ ] 50_leet
    - [x] test
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Ori_Data/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_leet/model-best/test_result.json
    ```
    <br>
    - [x] 10
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/10_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_leet/model-best/test_10_leet_result.json
    ```
    <br>
    - [x] 25
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/25_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_leet/model-best/test_25_leet_result.json
    ```
    <br>
    - [x] 50
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/50_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_leet/model-best/test_50_leet_result.json
    ```
    <br>
    - [x] 75
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/75_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_leet/model-best/test_75_leet_result.json
    ```
    <br>
    - [x] 100
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/100_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_leet/model-best/test_100_leet_result.json
    ```
    <br>

### 75 leet

- [ ] 75_leet
    - [x] test
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Ori_Data/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_leet/model-best/test_result.json
    ```
    <br>
    - [x] 10
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/10_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_leet/model-best/test_10_leet_result.json
    ```
    <br>
    - [x] 25
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/25_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_leet/model-best/test_25_leet_result.json
    ```
    <br>
    - [x] 50
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/50_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_leet/model-best/test_50_leet_result.json
    ```
    <br>
    - [x] 75
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/75_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_leet/model-best/test_75_leet_result.json
    ```
    <br>
    - [x] 100
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/100_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_leet/model-best/test_100_leet_result.json
    ```
    <br>

### 100 leet

- [ ] 100_leet
    - [x] test
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Ori_Data/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_leet/model-best/test_result.json
    ```
    <br>
    - [x] 10
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/10_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_leet/model-best/test_10_leet_result.json
    ```
    <br>
    - [x] 25
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/25_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_leet/model-best/test_25_leet_result.json
    ```
    <br>
    - [x] 50
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/50_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_leet/model-best/test_50_leet_result.json
    ```
    <br>
    - [ ] 75
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/75_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_leet/model-best/test_75_leet_result.json
    ```
    <br>
    - [ ] 100
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_leet/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/100_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_leet/model-best/test_100_leet_result.json
    ```
    <br>

### 10_augmented

- [ ] 10_augmented
    - [ ] test
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Ori_Data/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_augmented/model-best/test_result.json
    ```
    <br>
    - [ ] 10
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/10_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_augmented/model-best/test_10_leet_result.json
    ```
    <br>
    - [ ] 25
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/25_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_augmented/model-best/test_25_leet_result.json
    ```
    <br>
    - [ ] 50
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/50_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_augmented/model-best/test_50_leet_result.json
    ```
    <br>
    - [ ] 75
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/75_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_augmented/model-best/test_75_leet_result.json
    ```
    <br>
    - [ ] 100
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/100_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_10_augmented/model-best/test_100_leet_result.json
    ```
    <br>

### 25_augmented

- [ ] 25_augmented
    - [ ] test
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Ori_Data/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_augmented/model-best/test_result.json
    ```
    <br>
    - [ ] 10
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/10_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_augmented/model-best/test_10_leet_result.json
    ```
    <br>
    - [ ] 25
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/25_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_augmented/model-best/test_25_leet_result.json
    ```
    <br>
    - [ ] 50
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/50_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_augmented/model-best/test_50_leet_result.json
    ```
    <br>
    - [ ] 75
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/75_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_augmented/model-best/test_75_leet_result.json
    ```
    <br>
    - [ ] 100
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/100_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_25_augmented/model-best/test_100_leet_result.json
    ```
    <br>

### 50_augmented

- [ ] 50_augmented
    - [ ] test
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Ori_Data/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_augmented/model-best/test_result.json
    ```
    <br>
    - [ ] 10
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/10_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_augmented/model-best/test_10_leet_result.json
    ```
    <br>
    - [ ] 25
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/25_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_augmented/model-best/test_25_leet_result.json
    ```
    <br>
    - [ ] 50
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/50_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_augmented/model-best/test_50_leet_result.json
    ```
    <br>
    - [ ] 75
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/75_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_augmented/model-best/test_75_leet_result.json
    ```
    <br>
    - [ ] 100
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/100_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_50_augmented/model-best/test_100_leet_result.json
    ```
    <br>

### 75_augmented

- [ ] 75_augmented
    - [ ] test
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Ori_Data/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_augmented/model-best/test_result.json
    ```
    <br>
    - [ ] 10
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/10_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_augmented/model-best/test_10_leet_result.json
    ```
    <br>
    - [ ] 25
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/25_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_augmented/model-best/test_25_leet_result.json
    ```
    <br>
    - [ ] 50
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/50_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_augmented/model-best/test_50_leet_result.json
    ```
    <br>
    - [ ] 75
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/75_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_augmented/model-best/test_75_leet_result.json
    ```
    <br>
    - [ ] 100
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/100_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_75_augmented/model-best/test_100_leet_result.json
    ```
    <br>

### 100_augmented



- [ ] 100_augmented
    - [ ] test
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Ori_Data/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_augmented/model-best/test_result.json
    ```
    <br>
    - [ ] 10
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/10_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_augmented/model-best/test_10_leet_result.json
    ```
    <br>
    - [ ] 25
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/25_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_augmented/model-best/test_25_leet_result.json
    ```
    <br>
    - [ ] 50
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/50_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_augmented/model-best/test_50_leet_result.json
    ```
    <br>
    - [ ] 75
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/75_per/test.spacy  --gpu-id 0 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_augmented/model-best/test_75_leet_result.json
    ```
    <br>
    - [ ] 100
    ```bash
    python -m spacy evaluate /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_augmented/model-best /home/alvaro/data/WordCamouflage_Resiliance/Spacy_Data/Constraint/Leet_Data/100_per/test.spacy  --gpu-id 1 --output /home/alvaro/data/WordCamouflage_Resiliance/output_models/Constraint/bert-base-uncased_Constraint_100_augmented/model-best/test_100_leet_result.json
    ```
    <br>